In [46]:
import pandas as pd
import numpy as np

from ortools.sat.python import cp_model

In [56]:
data_file = "data/gc_500_1"
data = pd.read_csv(data_file, sep=" ", names=["vertices", "edges"])

In [57]:
data.head()

,vertices,edges
0,500,12565
1,0,27
2,0,28
3,0,53
4,0,55


In [58]:
n = data.vertices[0]
e = data.edges[0]
edge_list = np.array(data[1:])

In [59]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
  """Print intermediate solutions."""

  def __init__(self, variables):
    self.__variables = variables
    self.__solution_count = 0
    self.__solutions = []

  def NewSolution(self):
    self.__solution_count += 1
    self.__solutions.append([self.Value(v) for v in self.__variables])

  def SolutionCount(self):
    return self.__solution_count

  def GetSolutions(self):
    return self.__solutions

In [60]:
degrees = np.zeros((n,1))
for edge in edge_list:
    e1 = edge[0]
    e2 = edge[1]
    degrees[e1] += 1
    degrees[e2] += 1
max_degree_node = np.argmax(degrees)

In [ ]:
max_colours = 30
min_colours = 5
MAX_TIME = 60.0

for num_colours in range(min_colours, max_colours + 1):

    model = cp_model.CpModel()
    solver = cp_model.CpSolver()

    # Sets a time limit of 10 seconds.
    solver.parameters.max_time_in_seconds = MAX_TIME

    # Create the variables
    C = []
    for i in range(0, n):
        C.append(model.NewIntVar(0, num_colours - 1, "c_" + str(i)))

    # Create the constraints
    for edge in edge_list:
        e1 = edge[0]
        e2 = edge[1]
        model.Add(C[e1] != C[e2])

    # symmetry breaking
    # Chose vertex with highest degreee and assign it the first colour
    model.Add(C[max_degree_node] == 0)
    
    for i in range(1, num_colours):
        model.Add(C[i] <= i+1);

    model.Minimize(max(C))

    # Call the solver.
    solution_printer = SolutionPrinter(C)
    status = solver.SolveWithSolutionObserver(model, solution_printer)
    #print("Number of colours: %i" % num_colours)
    print('Number of solutions found: %i' % solution_printer.SolutionCount())
    
    if solution_printer.SolutionCount() > 0:
        break
        
# Now ensure you get solution with minimum
solutions = solution_printer.GetSolutions()
max_color = [max(C) for C in solutions]
best_index = np.argmin(max_color)
node_count = max_color[best_index]
solution = solutions[best_index]

Number of solutions found: 0
Number of solutions found: 0


In [55]:
node_count

4

In [29]:
num_colours

26

In [7]:
min_colours = 2 if n <= 4 else 3

In [8]:
min_colours

3

In [45]:
max_colours = 6
MAX_TIME = 300.0
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# Sets a time limit of 10 seconds.
solver.parameters.max_time_in_seconds = MAX_TIME

# Create the variables
C = []
for i in range(0, n):
    C.append(model.NewIntVar(0, max_colours - 1, "c_" + str(i)))

# Create the constraints
for edge in edge_list:
    e1 = edge[0]
    e2 = edge[1]
    model.Add(C[e1] != C[e2])

# symmetry breaking
# Chose vertex with highest degreee and assign it the first colour
model.Add(C[max_degree_node] == 0)

model.Minimize(max(C))


# Call the solver.
solution_printer = SolutionPrinter(C)
status = solver.SolveWithSolutionObserver(model, solution_printer)
#print("Number of colours: %i" % num_colours)
print('Number of solutions found: %i' % solution_printer.SolutionCount())

Number of solutions found: 0


In [44]:
solutions = solution_printer.GetSolutions()
for i in range(0, len(solutions)):
    print(max(solutions[i]))



In [11]:
??model.Minimize

In [30]:
solutions[0]

[1,
 2,
 0,
 0,
 1,
 0,
 3,
 1,
 1,
 0,
 4,
 1,
 2,
 1,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 3,
 0,
 2,
 1,
 1,
 3,
 1,
 3,
 0,
 2,
 0,
 0,
 2,
 2,
 3,
 0,
 0,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 3,
 2,
 3,
 2,
 2,
 4,
 2,
 4,
 0,
 3,
 4,
 3,
 4,
 4,
 2,
 3,
 3,
 4,
 4,
 4]